# Adaptive Boosting (Adaboost)

Adaboost is an ensemble learning method that combine multiple weak classifiers **(Stump)** to build one strong classifier. It is to iteratively train a sequence of stumps, each of which **focus on misclassified instance** classified by previous stump

**Step 1: Assign a sample weight for each sample <br>**
sample weight = $\frac{1}{N}$ <br>

**Step 2: Do a boostrap aggregating (bagging)**

**Step 3: Calculate the Gini Impurity for each variable <br>**
Gini Impurity for a Leaf = 1 - [\(probability of 'Yes'\)$^2$ + \(probability of 'No'\)$^2$] : $Gini=1 - \sum\limits_{i-1}^{n}(p_i)^2$ <br>
The variable with **lowest** Gini Impurity will be used to split the data

**Step 4: Build the stump** <br>
Use the variable with **lowest** Gini Impurity to build a stump

**Step 4: Use the stump to predict the train data**
calcuate the total error which is the sum of weights of **incorrectly** classified samples <br>
total error = $\sum weights$

**Step 5: Calculate the Amount of say of each stump** <br>
It is the importance / weight of a stump <br>
***this weight is the importance of stump which is different from the sample weight assigned initially <br>
_Amount of say_ = $\frac{1}{2} \log(\frac{1 - \text{total error}}{\text{total error}})$ <br>
_The higher, the more important_

**Step 6: Update the sample weights** <br>
$\text{Incorrect sample} = \text{Sample weight} * e^{\text{amount of say}}$ <br>
$\text{Correct sample} = \text{Sample weight} * e^{-\text{amount of say}}$

Remember to normalize the sample weights since the sum up is not equal to 1 <br>
$\text{normalized sample weight} = \frac{\text{updated sample weight}}{\sum\text{updated sample weight}}$
